In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

#Definimos una funcion para tratar a las imagenes (en vez de una por una como hice en el commit pasado):
def load_image(path, size=(64,64)):
    img = Image.open(path).resize(size)
    arr = np.array(img) / 255.0     # normalizar
    return arr.flatten().reshape(-1,1)  # vector columna

#Tomamos nuestras imagenes y les aplicamos la funcion:
x1 = load_image("Img/Dog1.jpg")
x2 = load_image("Img/Dog2.jpg")
x3 = load_image("Img/Dog3.jpg")
x4 = load_image("Img/Cat1.jpeg")
x5 = load_image("Img/Cat2.jpg")
x6 = load_image("Img/Cat3.jpg")

# Nuestras X son:
X = np.concatenate([x1, x2, x3, x4, x5, x6], axis=1)
print("X shape:", X.shape) 

# Etiquetas con perros=0 y gatos=1:
Y = np.array([[0, 0, 0, 1, 1, 1]])  
print("Y shape:", Y.shape) 

#Ahora si procedemos a inicializar los datos:
np.random.seed(1) #Con esto fijamos un numero aleatorio para evitar que cambie cada que se corra el codigo
n_x = X.shape[0]  #Para extraer los pixeles que tiene cada imagen
w = np.random.randn(n_x, 1) * 0.01 #Le doy numeros chicos para evitar sobrecarga en el calculo
b = 0
a= 0.1               # (es la tasa de aprendizaje)
i = 1000             # numero de iteraciones



X shape: (12288, 6)
Y shape: (1, 6)
